In [1]:
from fishsense_api_sdk.client import Client
from sklearn.cluster import HDBSCAN

In [2]:
URL = "http://localhost:8000"
DIVE_ID = 347

HDBSCAN_MIN_SAMPLES=2

In [3]:
client = Client(URL)

In [4]:
dive = await client.dives.get(dive_id=DIVE_ID)

dive

Dive(id=347, name='031424_Alligator0_FSL02', path='2025-02-10 REEF Data Dump SMILE 6/031424_Alligator_FSL02/031424_Alligator0_FSL02', dive_datetime=datetime.datetime(2024, 3, 14, 12, 15, 46, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=2, dive_slate_id=None)

In [5]:
images = await client.images.get(dive_id=dive.id)

len(images), images[0]

(333,
 Image(id=102731, path='2025-02-10 REEF Data Dump SMILE 6/031424_Alligator_FSL02/031424_Alligator0_FSL02/P3140042.ORF', taken_datetime=datetime.datetime(2024, 3, 14, 11, 19, 48, tzinfo=TzInfo(0)), checksum='56975f26d2e2c1281fda306fa95e7baf', is_canonical=True, dive_id=347, camera_id=2))

In [6]:
timestamps = [
    img.taken_datetime.timestamp() if img.taken_datetime else 0
    for img in images
]

len(timestamps), timestamps[0]

(333, 1710415188.0)

In [7]:
X = [[ts] for ts in timestamps]

In [8]:
db = HDBSCAN(min_cluster_size=HDBSCAN_MIN_SAMPLES).fit(X)
labels = db.labels_

len(labels), len(set(labels)), labels[0]

(333, 89, np.int64(34))

In [9]:
clusters = {}
cluster_len = len(set(labels)) - (1 if -1 in labels else 0)
for label, img in zip(labels, images):
    if label == -1:
        label = cluster_len
        cluster_len += 1

    if label not in clusters:
        clusters[label] = []
    clusters[label].append(img)

clusters = list(clusters.values())

len(clusters), len(clusters[0]), clusters[0][0]

(100,
 3,
 Image(id=102731, path='2025-02-10 REEF Data Dump SMILE 6/031424_Alligator_FSL02/031424_Alligator0_FSL02/P3140042.ORF', taken_datetime=datetime.datetime(2024, 3, 14, 11, 19, 48, tzinfo=TzInfo(0)), checksum='56975f26d2e2c1281fda306fa95e7baf', is_canonical=True, dive_id=347, camera_id=2))

In [10]:
for cluster in clusters:
    image_ids = [img.id for img in cluster]

    await client.images.post_cluster(DIVE_ID, image_ids)